# Copy the volume to make it a bit bigger

In [1]:
from pathlib import Path
import numpy as np
import xarray as xr
import dask.array as da

In [2]:
volume_path = Path(
    "C:\scratch\synthoseis_example\seismic__2024.38842098_example\seismicCubes_cumsum_fullstack_2024.38842098.npy"
)

In [26]:
volume_numpy = np.load(volume_path)
volume_numpy = volume_numpy.astype(np.float16)

In [27]:
volume_numpy.shape

(300, 300, 1259)

## Create some coordinates for xarray

In [28]:
track_coords = np.linspace(
    start=1000,
    stop=4000,
    num=300,
    dtype=np.uint32,
)

bin_coords = np.linspace(
    start=1000,
    stop=4000,
    num=300,
    dtype=np.uint32,
)

sample_coords = np.linspace(
    start=0.0,
    stop=5000.0,
    num=1259,
    dtype=np.float32,
)

dimensions = ("track_no", "bin_no", "depth")
coords = (track_coords, bin_coords, sample_coords)

In [29]:
volume_da = da.from_array(volume_numpy)

In [30]:
volume_da = xr.DataArray(
    data=volume_da,
    name="synthetic_seismic",
    coords=coords,
    dims=dimensions
)

In [46]:
copies = [volume_da for _ in range(10)]

In [47]:
stacked = da.concatenate(copies, axis=0)

In [48]:
new_copies = [stacked for _ in range(10)]

In [49]:
final_stacked = da.concatenate(new_copies, axis=1)

In [51]:
final_stacked = final_stacked.rechunk(chunks=(300, 300, "auto"))

In [52]:
final_stacked

dask.array<concatenate, shape=(3000, 3000, 1259), dtype=float16, chunksize=(300, 300, 745), chunktype=numpy.ndarray>

In [53]:
track_coords = np.linspace(
    start=1000,
    stop=4000,
    num=final_stacked.shape[0],
    dtype=np.uint32,
)

bin_coords = np.linspace(
    start=1000,
    stop=4000,
    num=final_stacked.shape[1],
    dtype=np.uint32,
)

sample_coords = np.linspace(
    start=0.0,
    stop=5000.0,
    num=final_stacked.shape[2],
    dtype=np.float32,
)

dimensions = ("track_no", "bin_no", "depth")
coords = (track_coords, bin_coords, sample_coords)

In [54]:
volume_da_stacked = xr.DataArray(
    data=final_stacked, name="synthetic_seismic", coords=coords, dims=dimensions
)

In [56]:
future_save = volume_da_stacked.to_zarr("s3://coiled-test-data-synthetic-seismic/synthetic_volume/", compute=False)

In [58]:
from dask.diagnostics import ProgressBar

In [59]:
with ProgressBar():
  future_save.compute()

[########################################] | 100% Completed | 20m 41s
